# Total Runs Model -  Random Forest

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn import metrics

pd.set_option('display.max_columns', 2105)
pd.set_option('display.max_rows',500)

In [2]:
games = pd.read_csv('../Data/MLBgames.csv')

In [ ]:
# taking the last years data for validation data
val_data = 

In [3]:
X = games[['v_runs', 'h_runs', 'total_outs', 'v_abs','v_hits','v_2b','v_3b','v_hr','v_rbis','v_sacbunt','v_sacfly','v_hbp','v_bb','v_ibb','v_k','v_sb',
           'v_cs','v_gidp','v_ci','v_lob','v_p_used','v_ind_er','v_team_er','v_wildp','v_balk','v_putouts','v_assts','v_errors','v_passed_ball','v_dp',
           'v_trip','h_abs','h_hits','h_2b','h_3b','h_hr','h_rbis','h_sacbunt','h_sacfly','h_hbp','h_bb','h_ibb','h_k','h_sb','h_cs','h_gidp','h_ci',
           'h_lob','h_p_used','h_ind_er','h_team_er','h_wildp','h_balk','h_putouts','h_assts','h_errors','h_passed_ball','h_dp','h_trip','run_diff', 
           'W', 'L']]
y = games['total']

In [4]:
categorical_columns = ['W', 'L']

# Create a ColumnTransformer with one-hot encoding
ct = ColumnTransformer(
    transformers=[
        ('oh', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_columns)
    ],
    remainder='passthrough'
)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_ct_train = ct.fit_transform(X_train)
X_ct_test = ct.transform(X_test)

In [8]:
model = RandomForestRegressor()
model.fit(X_ct_train, y_train)

RandomForestRegressor()

In [9]:
model.score(X_ct_test, y_test)

0.9997287152945679

In [10]:
preds = model.predict(X_ct_test)
preds

array([13.,  8., 15., ..., 13.,  7.,  8.])

In [11]:
cv_scores = cross_val_score(model, X_ct_train, y_train, cv=3)
cv_scores

array([0.99936807, 0.99948619, 0.99973178])

In [12]:
np.mean(cv_scores)

0.9995286776142619